In [54]:
%load_ext autoreload
%autoreload 2
import pygame
import os
import numpy as np
from datetime import datetime

from stable_baselines3 import PPO,SAC
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv,VecMonitor
from gymnasium.wrappers import FlattenObservation,TimeLimit,NormalizeObservation
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback,BaseCallback

from deform_rl.sim.Rectangle_env.environment import Rectangle1D

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
from gymnasium import ObservationWrapper
from gymnasium.spaces import Box,Dict

class CustomNormalizeObsrvation(ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.width = env.width
        self.height = env.height
        # self.observation_space = Box(low=np.array([0.,0.,-1000.,-1000.,0.,0.]), high=np.array([800.,800.,1000.,1000.,800.,800.]), shape=(6,), dtype=np.float64)
        self.observation_space = Box(low=-1, high=1, shape=(4,), dtype=np.float64)
    # def observation(self, observation):
    #     mean = np.array([self.width, self.height]) / 2
    #     position = (observation['position'])
    #     velocity = observation['velocity']
    #     target = (observation['target'])    
    # return np.concatenate([position, velocity, target])
    def observation(self, observation):
        # mean = np.array([self.width,self.height]) / 2
        position = observation['position']
        target = observation['target']
        velocity = observation['velocity']
        rel_target = target - position
        rel_target /= np.array([self.width,self.height])
        velocity /= np.array([self.width,self.height])

        return np.concatenate([rel_target,velocity])



In [56]:
class TensorboardCallback(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)
    def _on_step(self) -> bool:
        self
        
        

In [57]:
sim_cfg = {
    'width': 800,
    'height': 800,
    'FPS': 60,
    'gravity': 0,
    'damping': .15,
    'collision_slope': 0.01,
}
save_dir = os.path.join("saved_models")
log_dir = os.path.join("logs")
os.makedirs(save_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

def _init(threshold=30,seed=None):
    # Base env
    env = Rectangle1D(sim_config=sim_cfg, threshold=threshold, oneD=False, render_mode='human', seed=seed)
    env = CustomNormalizeObsrvation(env)
    # Apply wrappers
    # env = FlattenObservation(env)
    env = TimeLimit(env, max_episode_steps=1000)
    check_env(env, warn=True)

    return env


env = _init()
# env = VecMonitor(DummyVecEnv([_init]*4))

# eval_env = VecMonitor(DummyVecEnv([_init]))
eval_env = _init()
eval_callback = EvalCallback(
    eval_env=eval_env,
    n_eval_episodes=15,
    eval_freq=10000,
    best_model_save_path=save_dir,
    verbose=1,
    render=False
)



In [58]:
class LinearAgent:
    def __init__(self):
        pass
    # def predict(self, obs, deterministic=True):
    #     pos = obs[:2]
    #     target = obs[4:]
    #     diff = target - pos
    #     return diff/np.linalg.norm(diff), None
    def predict(self, obs, deterministic=True):
        rel_target = obs[:2]
        # target = obs[4:]
        # diff = target - pos
        return rel_target/np.linalg.norm(rel_target), None

tenv = _init(seed=60)
obs, _ = tenv.reset()
# tenv = eval_env

cnt = 0
rev_sum = 0
la = LinearAgent()
EP_CNT = 10
ep_cnt = 0
for i in range(10000):
    if cnt >= 1000:
        print("Killed by timeout")
        obs,_ = tenv.reset()
        cnt = 0
    action,_ = la.predict(obs, deterministic=True)
    obs, reward, done,truncated, info = tenv.step(action)
    rev_sum += reward
    tenv.render()
    if done:
        obs,_ = tenv.reset()
        ep_cnt += 1
        print("Episode done: ", cnt, "Reward: ", rev_sum)
        cnt=0
        rev_sum = 0
        if ep_cnt >= EP_CNT:
            break
    if pygame.event.get(pygame.QUIT):
        break
    cnt +=1
tenv.close()


Episode done:  59 Reward:  -260.1273690727733


In [51]:
# env.reset()
# for i in range(10):
#     act =env.action_space.sample()
#     obs, reward, done,truncated, info = env.step(act)
#     print(obs)
#     print(reward)
#     print(done)
#     print(truncated)
#     print(info)



In [52]:
model = PPO("MlpPolicy",env,device='cpu',verbose=0,tensorboard_log="./logs/").learn(500000,tb_log_name="test_run",callback=eval_callback)


KeyboardInterrupt: 

In [59]:
# env = VecMonitor(DummyVecEnv([_init]))

model2 = SAC("MlpPolicy",env,device='cpu',verbose=0,tensorboard_log="./logs/").learn(500000,tb_log_name="test_run_SAC",callback=eval_callback)

/home/michal/Documents/Skola/bakalarka/RL/RL-cable/.venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=-10268.87 +/- 24757.20
Episode length: 514.93 +/- 398.23
New best mean reward!
Eval num_timesteps=20000, episode_reward=-479.49 +/- 1017.03
Episode length: 305.53 +/- 231.97
New best mean reward!
Eval num_timesteps=30000, episode_reward=52.70 +/- 122.21
Episode length: 189.07 +/- 164.52
New best mean reward!
Eval num_timesteps=40000, episode_reward=-29.46 +/- 130.06
Episode length: 106.47 +/- 32.53
Eval num_timesteps=50000, episode_reward=85.34 +/- 110.98
Episode length: 107.20 +/- 48.62
New best mean reward!
Eval num_timesteps=60000, episode_reward=130.48 +/- 76.02
Episode length: 92.73 +/- 26.13
New best mean reward!
Eval num_timesteps=70000, episode_reward=163.63 +/- 75.37
Episode length: 86.00 +/- 23.78
New best mean reward!
Eval num_timesteps=80000, episode_reward=208.35 +/- 57.85
Episode length: 89.67 +/- 31.77
New best mean reward!
Eval num_timesteps=90000, episode_reward=224.78 +/- 44.42
Episode length: 95.67 +/- 43.38
New best mean rewa

KeyboardInterrupt: 

In [ ]:

model.learn(500000,tb_log_name="test_run",callback=eval_callback,reset_num_timesteps=False)

Eval num_timesteps=510720, episode_reward=181.69 +/- 69.13
Episode length: 81.07 +/- 27.84


KeyboardInterrupt: 

In [70]:
# random pick actions and visualize
tenv = _init(seed=605)
obs, _ = tenv.reset()
# tenv = eval_env

# t_model = model
t_model = SAC.load(os.path.join(save_dir, "best_model.zip"),force_reset=True,device='cpu')
EP_CNT = 10
ep_cnt = 0
cnt = 0
rev_sum = 0
for i in range(10000):
    if cnt >= 1000:
        print("Killed by timeout")
        obs,_ = tenv.reset()
        cnt = 0
    action,_ = t_model.predict(obs, deterministic=True)
    obs, reward, done,truncated, info = tenv.step(action)
    rev_sum += reward
    tenv.render()
    if done:
        obs,_ = tenv.reset()
        ep_cnt += 1
        print("Episode done: ", cnt, "Reward: ", rev_sum)
        cnt=0
        rev_sum = 0
        if ep_cnt >= EP_CNT:
            break
    if pygame.event.get(pygame.QUIT):
        break
    cnt +=1
tenv.close()

Episode done:  86 Reward:  234.97774542417386
Episode done:  123 Reward:  287.90880256851756
Episode done:  130 Reward:  130.07572510087448
Episode done:  127 Reward:  129.73621208657983
Episode done:  100 Reward:  196.6136509083163
